![image.png](attachment:image.png)

##### **Good luck with taking your exam. Keep working and make your dreams all come true. Seeing the results of all of your hard work will make this struggle worth it. We’re all thinking of you.** 
<b><font color='blue'>AI-PRO Spark Team ITI</font></b>

# NLP Using PySpark

## Objective:
- The objective from this project is to create a <b>Spam filter using NaiveBayes classifier</b>.
- It is required to obtain <b>f1_scored > 0.9</b>.
- We'll use a dataset from UCI Repository. SMS Spam Detection: https://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection
- Data is also provided for you in the assignment (you do not have to download it).

## To perform this task follow the following guiding steps:

### Create a spark session and import the required libraries

In [1]:
# importing libraries
from pyspark import SparkContext
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Final').getOrCreate()

### Read the readme file to learn more about the data

### Read the data into a DataFrame

In [2]:
path="/content/SMSSpamCollection"
df = spark.read.format("csv").option("delimiter", "\t").option("inferSchema",True).load(path)

In [3]:
df.show()

+----+--------------------+
| _c0|                 _c1|
+----+--------------------+
| ham|Go until jurong p...|
| ham|Ok lar... Joking ...|
|spam|Free entry in 2 a...|
| ham|U dun say so earl...|
| ham|Nah I don't think...|
|spam|FreeMsg Hey there...|
| ham|Even my brother i...|
| ham|As per your reque...|
|spam|WINNER!! As a val...|
|spam|Had your mobile 1...|
| ham|I'm gonna be home...|
|spam|SIX chances to wi...|
|spam|URGENT! You have ...|
| ham|I've been searchi...|
| ham|I HAVE A DATE ON ...|
|spam|XXXMobileMovieClu...|
| ham|Oh k...i'm watchi...|
| ham|Eh u remember how...|
| ham|Fine if thats th...|
|spam|England v Macedon...|
+----+--------------------+
only showing top 20 rows



### Print the schema

In [4]:
df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)



### Rename the first column to 'class' and second column to 'text'

In [5]:
df=df.withColumnRenamed("_c0", 'class')
df=df.withColumnRenamed("_c1", 'text')
df.printSchema()

root
 |-- class: string (nullable = true)
 |-- text: string (nullable = true)



### Show the first 10 rows from the dataframe
- Show once with truncate=True and once with truncate=False

In [6]:
df.show(10,truncate=False)

+-----+----------------------------------------------------------------------------------------------------------------------------------------------------------------+
|class|text                                                                                                                                                            |
+-----+----------------------------------------------------------------------------------------------------------------------------------------------------------------+
|ham  |Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...                                                 |
|ham  |Ok lar... Joking wif u oni...                                                                                                                                   |
|spam |Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075o

## Clean and Prepare the Data

### Create a new feature column contains the length of the text column

In [7]:
from pyspark.sql.functions import *
df= df.withColumn('length',length(df.text))

### Show the new dataframe

In [8]:
df.show()

+-----+--------------------+------+
|class|                text|length|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
|  ham|U dun say so earl...|    49|
|  ham|Nah I don't think...|    61|
| spam|FreeMsg Hey there...|   147|
|  ham|Even my brother i...|    77|
|  ham|As per your reque...|   160|
| spam|WINNER!! As a val...|   157|
| spam|Had your mobile 1...|   154|
|  ham|I'm gonna be home...|   109|
| spam|SIX chances to wi...|   136|
| spam|URGENT! You have ...|   155|
|  ham|I've been searchi...|   196|
|  ham|I HAVE A DATE ON ...|    35|
| spam|XXXMobileMovieClu...|   149|
|  ham|Oh k...i'm watchi...|    26|
|  ham|Eh u remember how...|    81|
|  ham|Fine if thats th...|    56|
| spam|England v Macedon...|   155|
+-----+--------------------+------+
only showing top 20 rows



### Get the average text length for each class (give alias name to the average length column)

In [9]:
df.groupby("class").agg(avg("length").alias("the average length")).show()

+-----+------------------+
|class|the average length|
+-----+------------------+
|  ham| 71.45431945307645|
| spam| 138.6706827309237|
+-----+------------------+



## Feature Transformations

### In this part you transform you raw text in to tf_idf model :
- For more information about TF-IDF check the following link: <b>(Not needed for the test)</b>
https://en.wikipedia.org/wiki/Tf%E2%80%93idf

### Perform the following steps to obtain TF-IDF:
1. Import the required transformers/estimators for the subsequent steps.
2. Create a <b>Tokenizer</b> from the text column.
3. Create a <b>StopWordsRemover</b> to remove the <b>stop words</b> from the column obtained from the <b>Tokenizer</b>.
4. Create a <b>CountVectorizer</b> after removing the <b>stop words</b>.
5. Create the <b>TF-IDF</b> from the <b>CountVectorizer</b>.

In [10]:
from pyspark.ml.feature import Tokenizer,StopWordsRemover, CountVectorizer,IDF,StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vector

In [11]:
tokenizer = Tokenizer(inputCol="text", outputCol="tokenized_text")
stop_remove = StopWordsRemover(inputCol='tokenized_text',outputCol='stop_tokens')
count_vec = CountVectorizer(inputCol='stop_tokens',outputCol='c_vec')
tf_idf = IDF(inputCol="c_vec", outputCol="tf_idf")

- Convert the <b>class column</b> to index using <b>StringIndexer</b>
- Create feature column from the <b>TF-IDF</b> and <b>lenght</b> columns.

In [12]:
ham_spam_numeric = StringIndexer(inputCol='class',outputCol='label')
clean_up = VectorAssembler(inputCols=['tf_idf','length'],outputCol='features')

## The Model
- Create a <b>NaiveBayes</b> classifier with the default parameters.

In [13]:
from pyspark.ml.classification import NaiveBayes
NaiveBayes_model = NaiveBayes()

## Pipeline
### Create a pipeline model contains all the steps starting from the Tokenizer to the NaiveBays classifier.

In [14]:
from pyspark.ml import Pipeline
df_pipe_NaiveBayes = Pipeline(stages=[tokenizer,stop_remove,count_vec,tf_idf,ham_spam_numeric,clean_up,NaiveBayes_model])


### Split your data to trian and test data with ratios 0.7 and 0.3 respectively.

In [15]:
(training,testing) = df.randomSplit([0.7,0.3],seed=42)


### Fit your Pipeline model to the training data

In [16]:
spam_predictor = df_pipe_NaiveBayes.fit(training)

### Perform predictions on tests dataframe

In [17]:
test_results = spam_predictor.transform(testing)
test_results.show()

+-----+--------------------+------+--------------------+--------------------+--------------------+--------------------+-----+--------------------+--------------------+--------------------+----------+
|class|                text|length|      tokenized_text|         stop_tokens|               c_vec|              tf_idf|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+------+--------------------+--------------------+--------------------+--------------------+-----+--------------------+--------------------+--------------------+----------+
|  ham| &lt;DECIMAL&gt; ...|   132|[, &lt;decimal&gt...|[, &lt;decimal&gt...|(11001,[3,81,128,...|(11001,[3,81,128,...|  0.0|(11002,[3,81,128,...|[-688.90165603252...|[1.0,1.6326453475...|       0.0|
|  ham| said kiss, kiss,...|   133|[, said, kiss,, k...|[, said, kiss,, k...|(11001,[3,115,200...|(11001,[3,115,200...|  0.0|(11002,[3,115,200...|[-654.35621509625...|[1.0,1.4746946214...|       0.0|


### Print the schema of the prediction dataframe

In [18]:
test_results.printSchema()

root
 |-- class: string (nullable = true)
 |-- text: string (nullable = true)
 |-- length: integer (nullable = true)
 |-- tokenized_text: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- stop_tokens: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- c_vec: vector (nullable = true)
 |-- tf_idf: vector (nullable = true)
 |-- label: double (nullable = false)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



## Model Evaluation
- Use <b>MulticlassClassificationEvaluator</b> to calculate the <b>f1_score</b>.

In [19]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
f1_eval = MulticlassClassificationEvaluator(metricName='f1')
f1 = f1_eval.evaluate(test_results)
print("Accuracy of model at predicting spam was: {}".format(f1))

Accuracy of model at predicting spam was: 0.9727502290227267


#we will use another model to compare results with each other

##we will logistic regression

In [20]:
# Import LogisticRegression
from pyspark.ml.classification import LogisticRegression

# Create a LogisticRegression Estimator
lr = LogisticRegression()

##Our metric will be areaUnderROC

In [21]:
# Import the evaluation submodule
import pyspark.ml.evaluation as evals

# Create a BinaryClassificationEvaluator
evaluator = evals.BinaryClassificationEvaluator(metricName="areaUnderROC")

##Next, you need to create a grid of values to search over when looking for the optimal hyperparameters

In [22]:
# Import the tuning submodule
import pyspark.ml.tuning as tune
import numpy as np
# Create the parameter grid
grid = tune.ParamGridBuilder()

# Add the hyperparameter
grid = grid.addGrid(lr.regParam, np.arange(0, .1, .01))
grid = grid.addGrid(lr.elasticNetParam, [0, 1])

# Build the grid
grid = grid.build()

In [23]:
from pyspark.ml import Pipeline
df_pipe_logistic = Pipeline(stages=[tokenizer,stop_remove,count_vec,tf_idf,ham_spam_numeric,clean_up,lr])

# Call data_prep_pipe_logistic.fit()
best_lr = df_pipe_logistic.fit(training)



In [24]:
# Use the model to predict the test set
test_results = best_lr.transform(testing)

# Evaluate the predictions
print(evaluator.evaluate(test_results))

0.9860881505061878


In [25]:
y_true = test_results.select(['label']).collect()
y_pred = test_results.select(['prediction']).collect()

In [26]:
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

         0.0       0.98      0.99      0.99      1381
         1.0       0.96      0.89      0.92       212

    accuracy                           0.98      1593
   macro avg       0.97      0.94      0.96      1593
weighted avg       0.98      0.98      0.98      1593



#The conclusion that logistic regression give us area under ROC curve =0.98
#which is so good as it close to 1 

# GOOD LUCK
<b><font color='GREEN'>AI-PRO Spark Team ITI</font></b>

![image-3.png](attachment:image-3.png)